In [1]:
import cv2 as cv
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from tqdm import tqdm
from torch import nn
import torch.nn.functional as F
import torch.nn.init as init
from torchvision.transforms import v2

In [2]:
from Functions import one_hot, accuracy, randint_distinct, augmant_data
from models import BaseCNNClassifier, ResNet18, ResNet34

In [ ]:
# Set the seed for reproducibility
torch.random.manual_seed(42)

In [ ]:
# Set the device to CUDA if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# Define a composition of transformations for data augmentation
transforms = v2.Compose([
    # Convert image to PIL Image format
    v2.ToPILImage(),
    # Randomly flip the image horizontally with 50% probability
    v2.RandomHorizontalFlip(0.5),
    # Randomly rotate the image by up to 45 degrees
    v2.RandomRotation(45),
    # Convert image to tensor format
    v2.ToTensor(),
])

In [ ]:
# Define constants for file names and image size
files_names = ['benign', 'malignant', 'normal']
image_sz = 224

# Initialize empty tensors for X and y
X = torch.tensor([], dtype=torch.float32)
y = torch.tensor([], dtype=torch.int16)

def collect_training_data(X, y):
    """
    Collect training data from files in the current directory.

    Args:
        X (torch.tensor): Empty tensor to store image data
        y (torch.tensor): Empty tensor to store class labels

    Returns:
        X (torch.tensor): Tensor with image data
        y (torch.tensor): Tensor with class labels
    """
    for file in files_names:
        # Construct path to file and get class number
        path = os.path.join(os.getcwd().replace('\\', '/'), file)
        class_num = files_names.index(file)
        print(f"Class: {file} | Class value: {class_num}")

        # Iterate over images in the file
        for img in tqdm(os.listdir(path)):
            # Skip images with "mask" in the name
            if "mask" not in img:
                # Read image and resize to image_sz
                img_array = cv.imread(os.path.join(path, img), cv.IMREAD_GRAYSCALE)
                new_array = cv.resize(img_array, (image_sz, image_sz))

                # Append image data to X and class label to y
                X = torch.cat((X, torch.tensor(new_array, dtype=torch.float32).unsqueeze(0)), dim=0)
                y = torch.cat((y, torch.tensor([class_num], dtype=torch.int16).unsqueeze(0)), dim=0)

    # Normalize image data and add channel dimension
    X = X / 256
    X = X.unsqueeze(1)

    return X, y

# Call the function to collect training data
X, y = collect_training_data(X, y)

In [7]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = X_train.to(device), X_test.to(device), y_train.to(device), y_test.to(device)

In [8]:
# Augment the data
X_temp, y_temp = augmant_data(X_train, y_train, transforms, 3)
X_train, y_train = torch.cat((X_temp, X_train), dim=0), torch.cat((y_temp, y_train), dim=0)
del X_temp, y_temp

In [9]:
# Initialize CNN model
model_0 = BaseCNNClassifier().to(device)

In [10]:
# Define Hyperparameters
batch_size = 32
learning_rate = 1e-4
num_epochs = 5000

# Define the loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer_0 = torch.optim.AdamW(model_0.parameters(), lr=learning_rate)

In [ ]:
# Define the training loop
for epoch in range(num_epochs):
    # Get random batches of data and one hot encode the labels
    batch_idxs = randint_distinct(0, X_train.size(0), batch_size)
    X_batch, y_batch = X_train[batch_idxs], y_train[batch_idxs]
    y_batch = one_hot(y_batch).to(device)

    # Forward pass, compute loss, compute accuracy
    y_pred = model_0(X_batch)
    loss = loss_fn(y_pred, y_batch)
    train_acc = accuracy(y_pred, y_batch)

    # Compute backpropagation and optimization
    optimizer_0.zero_grad()
    loss.backward()
    optimizer_0.step()

    # Evaluate the model on a test set every 10 epochs
    if (epoch+1) % 10 == 0:
        model_0.eval()
        with torch.inference_mode():
            # Get random batches of data and one hot encode the labels
            batch_idxs = randint_distinct(0, X_test.size(0), X_test.size(0))
            X_batch, y_batch = X_test[batch_idxs], y_test[batch_idxs]
            y_batch = one_hot(y_batch).to(device)

            # Forward pass, compute loss, compute accuracy
            test_logits = model_0(X_batch)
            test_loss = loss_fn(test_logits, y_batch)
            test_acc = accuracy(test_logits, y_batch)

        model_0.train()

        # Print the loss and accuracy
        print(f"Epoch: {epoch+1} | Train loss: {loss.item():.4f} | Train Acc: {train_acc.item():.4f} | Test loss: {test_loss.item():.4f} | Test Acc: {test_acc.item():.4f}\n")

        # Save the model if the test accuracy is higher than 0.8 and higher than the previous best accuracy
        if test_acc.item() > 0.8 and test_acc.item() > model_0.best_acc:
            # Get current model accuracy and index
            model_0.best_acc = test_acc.item()
            idxs = [int(file.split("Idx")[1][:-4]) for file in os.listdir(os.getcwd().replace('\\', '/') + "/saves") if "Idx" in file]
            idx = max(idxs) + 1 if idxs else 0
            
            # Save the model
            torch.save(model_0.state_dict(), os.getcwd().replace('\\', '/') + f"/saves/BaseCNNAcc{test_acc.item():.5f}Idx{idx}.pth")
            print(f"Model saved successfully with accuracy: {test_acc.item():.4f}")

# Print the best accuracy
print(f"Best Accuracy: {model_0.best_acc:.4f}")

In [12]:
# Initialize ResNet18 model
model_1 = ResNet18().to(device)

In [13]:
# Define Hyperparameters
batch_size = 32
learning_rate = 1e-4
num_epochs = 5000

# Define the loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model_1.parameters(), lr=learning_rate)

In [ ]:
# Define the training loop
for epoch in range(num_epochs):
    # Get random batches of data and one hot encode the labels
    batch_idxs = randint_distinct(0, X_train.size(0), batch_size)
    X_batch, y_batch = X_train[batch_idxs], y_train[batch_idxs]
    y_batch = one_hot(y_batch).to(device)

    # Forward pass, compute loss, compute accuracy
    y_pred = model_1(X_batch)
    loss = loss_fn(y_pred, y_batch)
    train_acc = accuracy(y_pred, y_batch)

    # Compute backpropagation and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Evaluate the model on a test set every 10 epochs
    if (epoch+1) % 10 == 0:
        model_1.eval()
        with torch.inference_mode():
            # Get random batches of data and one hot encode the labels
            batch_idxs = randint_distinct(0, X_test.size(0), X_test.size(0))
            X_batch, y_batch = X_test[batch_idxs], y_test[batch_idxs]
            y_batch = one_hot(y_batch).to(device)

            # Forward pass, compute loss, compute accuracy
            test_logits = model_1(X_batch)
            test_loss = loss_fn(test_logits, y_batch)
            test_acc = accuracy(test_logits, y_batch)
        
        model_1.train()

        # Print the loss and accuracy
        print(f"Epoch: {epoch+1} | Train loss: {loss.item():.4f} | Train Acc: {train_acc.item():.4f} | Test loss: {test_loss.item():.4f} | Test Acc: {test_acc.item():.4f}\n")

        # Save the model if the test accuracy is higher than 0.8 and higher than the previous best accuracy
        if test_acc.item() > 0.8 and test_acc.item() > model_1.best_acc:
            # Get current model accuracy and index
            model_1.best_acc = test_acc.item()
            idxs = [int(file.split("Idx")[1][:-4]) for file in os.listdir(os.getcwd().replace('\\', '/') + "/saves") if "Idx" in file]
            idx = max(idxs) + 1 if idxs else 0
            
            # Save the model
            torch.save(model_1.state_dict(), os.getcwd().replace('\\', '/') + f"/saves/ResNet18Acc{test_acc.item():.5f}Idx{idx}.pth")
            print(f"Model saved successfully with accuracy: {test_acc.item():.4f}")

# Print the best accuracy
print(f"Best accuracy: {model_1.best_acc:.4f}")

In [41]:
# Initialize ResNet34 model
model_2 = ResNet34().to(device)

In [42]:
# Define Hyperparameters
batch_size = 32
learning_rate = 1e-4
num_epochs = 5000

# Define the loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model_2.parameters(), lr=learning_rate)

In [ ]:
# Define the training loop
for epoch in range(num_epochs):
    # Get random batches of data and one hot encode the labels
    batch_idxs = randint_distinct(0, X_train.size(0), batch_size)
    X_batch, y_batch = X_train[batch_idxs], y_train[batch_idxs]
    y_batch = one_hot(y_batch).to(device)

    # Forward pass, compute loss, compute accuracy
    y_pred = model_2(X_batch)
    loss = loss_fn(y_pred, y_batch)
    train_acc = accuracy(y_pred, y_batch)

    # Compute backpropagation and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Evaluate the model on a test set every 10 epochs
    if (epoch+1) % 10 == 0:
        model_2.eval()
        with torch.inference_mode():
            # Get random batches of data and one hot encode the labels
            batch_idxs = randint_distinct(0, X_test.size(0), X_test.size(0))
            X_batch, y_batch = X_test[batch_idxs], y_test[batch_idxs]
            y_batch = one_hot(y_batch).to(device)

            # Forward pass, compute loss, compute accuracy
            test_logits = model_2(X_batch)
            test_loss = loss_fn(test_logits, y_batch)
            test_acc = accuracy(test_logits, y_batch)
        
        model_2.train()

        # Print the loss and accuracy
        print(f"Epoch: {epoch+1} | Train loss: {loss.item():.4f} | Train Acc: {train_acc.item():.4f} | Test loss: {test_loss.item():.4f} | Test Acc: {test_acc.item():.4f}\n")

        # Save the model if the test accuracy is higher than 0.8 and higher than the previous best accuracy
        if test_acc.item() > 0.8 and test_acc.item() > model_2.best_acc:
            # Get current model accuracy and index
            model_2.best_acc = test_acc.item()
            idxs = [int(file.split("Idx")[1][:-4]) for file in os.listdir(os.getcwd().replace('\\', '/') + "/saves") if "Idx" in file]
            idx = max(idxs) + 1 if idxs else 0

            # Save the model
            torch.save(model_2.state_dict(), os.getcwd().replace('\\', '/') + f"/saves/ResNet34Acc{test_acc.item():.5f}Idx{idx}.pth")
            print(f"Model saved successfully with accuracy: {test_acc.item():.4f}")

# Print the best accuracy
print(f"Best accuracy: {model_2.best_acc:.4f}")

Clean inaffective models

In [27]:
paths = os.listdir(os.getcwd().replace('\\', '/') + "/saves")
accs = [acc.split("Acc")[1] for acc in [path.split("Idx")[0] for path in paths]]
for i in range(len(accs)):
    if float(accs[i]) < 0.82:
        os.remove(os.getcwd().replace('\\', '/') + "/saves/" + paths[i])

In [28]:
paths = os.listdir(os.getcwd().replace('\\', '/') + "/saves")
accs = [acc.split("Acc")[1] for acc in [path.split("Idx")[0] for path in paths]]
for i in range(len(accs)):
    if float(accs[i]) < 0.86 and "ResNet18" in paths[i]:
        os.remove(os.getcwd().replace('\\', '/') + "/saves/" + paths[i])

In [29]:
paths = os.listdir(os.getcwd().replace('\\', '/') + "/saves")
accs = [acc.split("Acc")[1] for acc in [path.split("Idx")[0] for path in paths]]
for i in range(len(accs)):
    if float(accs[i]) < 0.84 and "ResNet34" in paths[i]:
        os.remove(os.getcwd().replace('\\', '/') + "/saves/" + paths[i])